In [1]:
import numpy as np
import pandas as pd

### Choose a time partition

In [56]:
tp = 4

Load Database

In [57]:
data  = pd.read_csv(f'datos\counted\d{tp}_counted.csv', index_col=0)

In [58]:
data.columns

Index(['avisoid', 'empresaid', 'avisofechapublicacion', 'avisovacante',
       'mostrarsueldo', 'avisoexperiencia', 'expiracion', 'dias',
       'avisorepublicacion', 'avisocargo', 'areanombre',
       'actividadempresanombre', 'avisocuerpo', 'disponibilidadnombre',
       'avisoduracioncont', 'avisolugartrabajo', 'gradoescolarnombre',
       'situacionestudios', 'avisorequisitos', 'carreras', 'estado',
       'endpagado_o_gratuito', 'nivelnombre', 'carreras_array',
       'sueldoestimado', '_merge', 'teletrabajo_', 'avcu_b1', 'avca_b1',
       'avre_b1', 'avcu_b2', 'avca_b2', 'avre_b2', 'avcu_b3', 'avca_b3',
       'avre_b3', 'avcu_b4', 'avca_b4', 'avre_b4'],
      dtype='object')

In [59]:
data['bucket1'] = (data['avca_b1'] | data['avcu_b1'] | data['avre_b1'])
data['bucket2'] = (data['avca_b2'] | data['avcu_b2'] | data['avre_b2'] | data['teletrabajo_'])
data['bucket3'] = (data['avca_b3'] | data['avcu_b3'] | data['avre_b3'])
data['bucket4'] = (data['avca_b4'] | data['avcu_b4'] | data['avre_b4'])
data['wfh'] = (data['bucket2'] | data['bucket3'] | data['bucket4'])

In [60]:
results = pd.read_excel('results.xlsx', index_col=0)

In [61]:
results['bucket1'][tp] = data['bucket1'].sum()
results['bucket2'][tp] = data['bucket2'].sum()
results['bucket3'][tp] = data['bucket3'].sum()
results['bucket4'][tp] = data['bucket4'].sum()
results['wfh'][tp] = data['wfh'].sum()

C:\Users\OscarJaramillo\AppData\Local\Temp\ipykernel_26932\2926926372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['bucket1'][tp] = data['bucket1'].sum()
C:\Users\OscarJaramillo\AppData\Local\Temp\ipykernel_26932\2926926372.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['bucket2'][tp] = data['bucket2'].sum()
C:\Users\OscarJaramillo\AppData\Local\Temp\ipykernel_26932\2926926372.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

## Save results

In [62]:
data.to_csv(f'datos\counted\d{tp}_counted.csv')

In [63]:
results.to_excel('results.xlsx')

In [64]:
results

,avcu_bucket 1,avcu_bucket 2,avcu_bucket 3,avcu_bucket 4,avca_bucket 1,avca_bucket 2,avca_bucket 3,avca_bucket 4,avre_bucket 1,avre_bucket 2,avre_bucket 3,avre_bucket 4,teletrabajo,bucket1,bucket2,bucket3,bucket4,wfh
1,55701,7362,40488,5511,124,1032,188,130,7563,1725,2110,822,47,60657.0,8719.0,42008.0,6281.0,54246.0
2,3743,489,3173,191,2,55,9,4,242,47,56,20,37,3883.0,549.0,3208.0,211.0,3822.0
3,3722,3290,2933,362,3,419,266,6,260,537,253,49,10671,3870.0,11972.0,3204.0,404.0,13732.0
4,36048,30887,29172,3564,31,5898,3137,24,4802,6995,2447,177,37595,39133.0,58163.0,32701.0,3708.0,77842.0
